In [1]:
%matplotlib inline

import cv2 # The OpenCV library; install using `pip install opencv-contrib-python`
import numpy as np # Helpful when working with arrays; install using `pip install numpy`
from matplotlib import pyplot as plt # Good for graphing; install using `pip install matplotlib`
from matplotlib import image as image
import easygui # An easy-to-use file-picker; pip install easygui (mac not supported)

In [2]:
def get_mask(frame1, frame2, kernel=np.array((9,9), dtype=np.uint8)):
    
    frame_diff = cv2.subtract(frame2, frame1)

    # blur the frame difference
    frame_diff = cv2.medianBlur(frame_diff, 3)
    
    mask = cv2.adaptiveThreshold(frame_diff, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 11, 3)

    mask = cv2.medianBlur(mask, 3)

    # morphological operations
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel, iterations=1)

    return mask




In [3]:
def get_contour_detections(mask, thresh=400):
    # get mask contours
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_TC89_L1)
    detections = []
    for cnt in contours:
        x,y,w,h = cv2.boundingRect(cnt)
        area = w*h
        if area > thresh: 
            detections.append([x,y,x+w,y+h, area])

    return np.array(detections)

In [4]:
def remove_contained_bboxes(boxes):
    check_array = np.array([True, True, False, False])
    keep = list(range(0, len(boxes)))
    for i in keep: # range(0, len(bboxes)):
        for j in range(0, len(boxes)):
            # check if box j is completely contained in box i
            if np.all((np.array(boxes[j]) >= np.array(boxes[i])) == check_array):
                try:
                    keep.remove(j)
                except ValueError:
                    continue
    return keep


def non_max_suppression(boxes, scores, threshold=1e-1):
    # Sort the boxes by score in descending order
    boxes = boxes[np.argsort(scores)[::-1]]

    # remove all contained bounding boxes and get ordered index
    order = remove_contained_bboxes(boxes)

    keep = []
    while order:
        i = order.pop(0)
        keep.append(i)
        for j in order:
            # Calculate the IoU between the two boxes
            intersection = max(0, min(boxes[i][2], boxes[j][2]) - max(boxes[i][0], boxes[j][0])) * \
                           max(0, min(boxes[i][3], boxes[j][3]) - max(boxes[i][1], boxes[j][1]))
            union = (boxes[i][2] - boxes[i][0]) * (boxes[i][3] - boxes[i][1]) + \
                    (boxes[j][2] - boxes[j][0]) * (boxes[j][3] - boxes[j][1]) - intersection
            iou = intersection / union

            # Remove boxes with IoU greater than the threshold
            if iou > threshold:
                order.remove(j)
                
    return boxes[keep]

In [5]:
def get_detections(frame1, frame2, bbox_thresh=400, nms_thresh=1e-3, mask_kernel=np.array((9,9), dtype=np.uint8)):
    # get image mask for moving pixels
    mask = get_mask(frame1, frame2, mask_kernel)

    # get initially proposed detections from contours
    detections = get_contour_detections(mask, bbox_thresh)
    
    # Check if detections are not empty and have the expected shape
    if detections is None or len(detections) == 0:
        return np.array([])  # Return an empty array if there are no detections
    
    # Make sure detections is a 2D array
    detections = np.atleast_2d(detections)
    
    # separate bboxes and scores
    bboxes = detections[:, :4]  # First 4 columns should be bounding boxes
    scores = detections[:, -1]  # Last column should be scores

    # perform Non-Maximal Suppression on initial detections
    return non_max_suppression(bboxes, scores, nms_thresh)


In [6]:
video = cv2.VideoCapture('Images/traffic.mp4')

# Compute motion mask
kernel = np.array((9, 9), dtype=np.uint8)

while True:
    ret, frame1 = video.read()
    ret, frame2 = video.read()
    
    if not ret:
        break
    
    # Make a copy of the original colored frame for drawing rectangles
    display_frame = frame1.copy()
    
    # Convert frames to grayscale for motion detection
    frame1_gray = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
    frame2_gray = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)
    
    # Compute motion ma

    
    detections = get_detections(frame1_gray, frame2_gray, bbox_thresh=400, nms_thresh=1e-3, mask_kernel=kernel)
    
    # Draw rectangles around detected objects
    for x1, y1, x2, y2 in detections:
        cv2.rectangle(display_frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        
    # Display the frame with rectangles
    cv2.imshow('frame', display_frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cv2.destroyAllWindows()
video.release()


2024-10-09 15:25:51.848 python[60847:1291586] +[IMKClient subclass]: chose IMKClient_Legacy
2024-10-09 15:25:51.848 python[60847:1291586] +[IMKInputSession subclass]: chose IMKInputSession_Legacy


In [7]:
video = cv2.VideoCapture('Images/traffic.mp4')

# Compute motion mask
kernel = np.array((9, 9), dtype=np.uint8)

while True:
    ret, frame1 = video.read()
    ret, frame2 = video.read()
    
    if not ret:
        break
    
    # Make a copy of the original colored frame for drawing rectangles
    display_frame = frame1.copy()
    
    # Convert frames to grayscale for motion detection
    frame1_gray = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
    frame2_gray = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)
    
    # Compute motion mask
    mask = get_mask(frame1_gray, frame2_gray, kernel)
    detections = get_contour_detections(mask)
    
    # Draw green rectangles around detected objects on the colored frame
    for x1, y1, x2, y2, _ in detections:
        cv2.rectangle(display_frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        
    # Display the frame with rectangles
    cv2.imshow('frame', display_frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cv2.destroyAllWindows()
video.release()
